# Complexity Analysis Models in Composer AI

This notebook explores the sophisticated complexity analysis models used in the Composer AI system for assessing musical difficulty. We'll examine each component of the complexity assessment system and demonstrate how it works with real musical examples.

## Overview

The Composer AI system evaluates musical complexity across four main dimensions:

1. **Harmonic Complexity** - Chord variety, extensions, key changes
2. **Rhythmic Complexity** - Time signatures, tempo, rhythmic patterns
3. **Technical Complexity** - Voice leading, execution difficulty
4. **Melodic Complexity** - Interval patterns, voice movement

These are combined using a polynomial regression model to produce an overall difficulty score from 0-10, with corresponding skill level classifications.

In [ ]:
# Import required libraries
import os
import sys

# Add the src directory to the path so we can import composer
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("")), "src"))

import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from IPython.display import display
from plotly.subplots import make_subplots

import composer

# Set up plotting style
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")

print("✅ All imports successful!")
print(
    f"Composer library version: {composer.__version__ if hasattr(composer, '__version__') else 'Unknown'}"
)

## Initialize the AI Engine

First, let's set up the AI engine with some training data to enable complexity analysis.

In [ ]:
# Initialize the AI engine with default configuration
engine = composer.AiEngine()

# Create some sample training patterns for the engine
# These represent common chord progressions in different styles
training_patterns = [
    # Pop/Rock progressions (simple)
    (
        [
            composer.Chord(1, 5),
            composer.Chord(5, 5),
            composer.Chord(6, 5),
            composer.Chord(4, 5),
        ],
        "pop-I-V-vi-IV",
        "C",
    ),
    (
        [
            composer.Chord(1, 5),
            composer.Chord(6, 5),
            composer.Chord(4, 5),
            composer.Chord(5, 5),
        ],
        "pop-I-vi-IV-V",
        "C",
    ),
    # Jazz progressions (moderate to complex)
    (
        [composer.Chord(2, 7), composer.Chord(5, 7), composer.Chord(1, 7)],
        "jazz-ii-V-I",
        "C",
    ),
    (
        [
            composer.Chord(6, 7),
            composer.Chord(2, 7),
            composer.Chord(5, 7),
            composer.Chord(1, 7),
        ],
        "jazz-vi-ii-V-I",
        "C",
    ),
    # Classical progressions (moderate)
    (
        [
            composer.Chord(1, 5),
            composer.Chord(4, 5),
            composer.Chord(5, 7),
            composer.Chord(1, 5),
        ],
        "classical-I-IV-V7-I",
        "C",
    ),
    (
        [
            composer.Chord(1, 5),
            composer.Chord(6, 5),
            composer.Chord(2, 5),
            composer.Chord(5, 7),
        ],
        "classical-I-vi-ii-V7",
        "C",
    ),
]

# Initialize the engine with training data
engine.initialize(training_patterns)

print(f"✅ AI Engine initialized with {len(training_patterns)} training patterns")
print(
    f"Engine status: {'Initialized' if engine.is_initialized() else 'Not initialized'}"
)

## Sample Chord Progressions

Let's create a variety of chord progressions with different complexity levels to analyze:

In [ ]:
# Define test chord progressions with varying complexity
test_progressions = {
    "Beginner - Simple Triads": {
        "chords": [
            composer.Chord(1, 5),
            composer.Chord(4, 5),
            composer.Chord(5, 5),
            composer.Chord(1, 5),
        ],
        "description": "Basic I-IV-V-I progression with simple triads",
        "tempo": 120.0,
        "time_sig": (4, 4),
    },
    "Intermediate - Seventh Chords": {
        "chords": [
            composer.Chord(1, 7),
            composer.Chord(6, 7),
            composer.Chord(2, 7),
            composer.Chord(5, 7),
        ],
        "description": "I7-vi7-ii7-V7 with all seventh chords",
        "tempo": 140.0,
        "time_sig": (4, 4),
    },
    "Advanced - Extended Harmonies": {
        "chords": [
            composer.Chord(1, 9),
            composer.Chord(6, 11),
            composer.Chord(2, 9),
            composer.Chord(5, 13),
        ],
        "description": "Extended harmonies: I9-vi11-ii9-V13",
        "tempo": 160.0,
        "time_sig": (7, 8),
    },
    "Expert - Complex Jazz": {
        "chords": [
            composer.Chord(1, 9),  # Imaj9
            composer.Chord(3, 7),  # iii7
            composer.Chord(6, 11),  # vi11
            composer.Chord(2, 9),  # ii9
            composer.Chord(5, 13),  # V13
            composer.Chord(1, 7),  # Imaj7
        ],
        "description": "Complex jazz progression with extended chords and chromatic movement",
        "tempo": 180.0,
        "time_sig": (5, 4),
    },
}

print("Test progressions created:")
for name, prog in test_progressions.items():
    chord_names = [f"Chord({c.root}, {c.chord_type})" for c in prog["chords"]]
    print(f"  {name}: {' -> '.join(chord_names)}")

## Complexity Analysis

Now let's analyze each progression and examine the complexity scores:

In [ ]:
# Analyze complexity for each test progression
complexity_results = {}

for name, prog in test_progressions.items():
    try:
        assessment = engine.assess_difficulty(
            prog["chords"], prog["tempo"], prog["time_sig"]
        )

        complexity_results[name] = {
            "overall_score": assessment.overall_score,
            "harmonic_complexity": assessment.harmonic_complexity,
            "rhythmic_complexity": assessment.rhythmic_complexity,
            "technical_complexity": assessment.technical_complexity,
            "melodic_complexity": assessment.melodic_complexity,
            "skill_level": assessment.skill_level,
            "confidence": assessment.confidence,
            "factors": {
                "unique_chords": assessment.unique_chords,
                "extended_harmonies": assessment.extended_harmonies,
                # Note: Other factor properties may not be available in the current API
                "avg_chord_complexity": 0.0,  # Placeholder - calculate manually if needed
                "key_changes": 0,  # Placeholder - not available in current API
                "voice_leading_complexity": 0.0,  # Placeholder - not available in current API
            },
            "description": prog["description"],
        }

        print(
            f"✅ {name}: Overall Score = {assessment.overall_score:.2f}, Skill Level = {assessment.skill_level}"
        )

    except Exception as e:
        print(f"❌ Error analyzing {name}: {e}")

print(f"\n📊 Successfully analyzed {len(complexity_results)} progressions")

## Visualization: Complexity Breakdown

Let's create comprehensive visualizations to understand how each complexity component contributes to the overall score:

In [ ]:
# Create DataFrame for easier plotting
df_complexity = pd.DataFrame(
    {
        "Progression": list(complexity_results.keys()),
        "Overall Score": [r["overall_score"] for r in complexity_results.values()],
        "Harmonic": [r["harmonic_complexity"] for r in complexity_results.values()],
        "Rhythmic": [r["rhythmic_complexity"] for r in complexity_results.values()],
        "Technical": [r["technical_complexity"] for r in complexity_results.values()],
        "Melodic": [r["melodic_complexity"] for r in complexity_results.values()],
        "Skill Level": [r["skill_level"] for r in complexity_results.values()],
        "Confidence": [r["confidence"] for r in complexity_results.values()],
    }
)

# Create interactive radar chart for complexity components
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=list(complexity_results.keys()),
    specs=[
        [{"type": "polar"}, {"type": "polar"}],
        [{"type": "polar"}, {"type": "polar"}],
    ],
)

categories = ["Harmonic", "Rhythmic", "Technical", "Melodic"]
colors = ["#FF6B6B", "#4ECDC4", "#45B7D1", "#96CEB4"]

for i, (name, result) in enumerate(complexity_results.items()):
    row = i // 2 + 1
    col = i % 2 + 1

    values = [
        result["harmonic_complexity"],
        result["rhythmic_complexity"],
        result["technical_complexity"],
        result["melodic_complexity"],
    ]

    fig.add_trace(
        go.Scatterpolar(
            r=values + [values[0]],  # Close the shape
            theta=categories + [categories[0]],
            fill="toself",
            fillcolor=colors[i],
            line=dict(color=colors[i], width=2),
            opacity=0.7,
            name=name,
        ),
        row=row,
        col=col,
    )

    # Update polar layout for each subplot
    fig.update_polars(
        radialaxis=dict(range=[0, 10], tickmode="linear", tick0=0, dtick=2),
        row=row,
        col=col,
    )

fig.update_layout(
    title="Complexity Analysis Breakdown by Component", height=800, showlegend=False
)

fig.show()

# Create bar chart comparing overall scores
fig_bar = px.bar(
    df_complexity,
    x="Progression",
    y="Overall Score",
    color="Skill Level",
    title="Overall Complexity Scores by Skill Level",
    color_discrete_map={
        "Beginner": "#96CEB4",
        "Intermediate": "#FECA57",
        "Advanced": "#FF9FF3",
        "Expert": "#FF6B6B",
    },
)

fig_bar.update_layout(
    xaxis_title="Chord Progression", yaxis_title="Complexity Score (0-10)", height=500
)

fig_bar.show()

## Detailed Factor Analysis

Let's examine the underlying factors that contribute to complexity scores:

In [ ]:
# Create detailed factor analysis
factor_data = []

for name, result in complexity_results.items():
    factors = result["factors"]
    factor_data.append(
        {
            "Progression": name,
            "Unique Chords": factors["unique_chords"],
            "Avg Chord Complexity": factors["avg_chord_complexity"],
            "Key Changes": factors["key_changes"],
            "Extended Harmonies": factors["extended_harmonies"],
            "Voice Leading Complexity": factors["voice_leading_complexity"],
            "Overall Score": result["overall_score"],
        }
    )

df_factors = pd.DataFrame(factor_data)

# Create heatmap of factors
fig, ax = plt.subplots(figsize=(12, 6))

# Normalize factors for better visualization
factor_cols = [
    "Unique Chords",
    "Avg Chord Complexity",
    "Key Changes",
    "Extended Harmonies",
    "Voice Leading Complexity",
]

df_factors_norm = df_factors.copy()
for col in factor_cols:
    max_val = df_factors[col].max()
    if max_val > 0:
        df_factors_norm[col] = df_factors[col] / max_val

# Create heatmap
heatmap_data = df_factors_norm.set_index("Progression")[factor_cols].T
sns.heatmap(
    heatmap_data,
    annot=True,
    cmap="YlOrRd",
    cbar_kws={"label": "Normalized Factor Value"},
    ax=ax,
    fmt=".2f",
)

ax.set_title("Complexity Factors Heatmap (Normalized)", fontsize=14, fontweight="bold")
ax.set_xlabel("Chord Progression", fontsize=12)
ax.set_ylabel("Complexity Factor", fontsize=12)

plt.tight_layout()
plt.show()

# Display raw factor values
print("\n📋 Raw Factor Values:")
print(df_factors.to_string(index=False))

## Polynomial Regression Model Analysis

The overall complexity score is calculated using a polynomial regression model. Let's analyze how the model combines different complexity components:

In [ ]:
# Model weights from the implementation (analysis.rs lines 456-480)
model_weights = {
    "harmonic_complexity": 0.35,
    "rhythmic_complexity": 0.25,
    "technical_complexity": 0.25,
    "melodic_complexity": 0.15,
}

# Polynomial coefficients (default from implementation)
coefficients = [1.0, 0.5, 4.0, 0.0]  # [a, b, c, d] for ax³ + bx² + cx + d

print("🔧 Model Configuration:")
print("Feature Weights:")
for factor, weight in model_weights.items():
    print(f"  {factor}: {weight:.0%}")
print(
    f"\nPolynomial Coefficients: a={coefficients[0]}, b={coefficients[1]}, c={coefficients[2]}, d={coefficients[3]}"
)


# Calculate weighted input for each progression
def calculate_weighted_input(harmonic, rhythmic, technical, melodic):
    return (
        harmonic * model_weights["harmonic_complexity"]
        + rhythmic * model_weights["rhythmic_complexity"]
        + technical * model_weights["technical_complexity"]
        + melodic * model_weights["melodic_complexity"]
    )


def apply_polynomial(weighted_input):
    x = weighted_input / 10.0  # Normalize input
    a, b, c, d = coefficients
    result = a * x**3 + b * x**2 + c * x + d
    return max(0.0, min(10.0, result * 10.0))  # Scale back and clamp


# Analyze the model for our progressions
model_analysis = []

for name, result in complexity_results.items():
    weighted_input = calculate_weighted_input(
        result["harmonic_complexity"],
        result["rhythmic_complexity"],
        result["technical_complexity"],
        result["melodic_complexity"],
    )

    predicted_score = apply_polynomial(weighted_input)
    actual_score = result["overall_score"]

    model_analysis.append(
        {
            "Progression": name,
            "Weighted Input": weighted_input,
            "Predicted Score": predicted_score,
            "Actual Score": actual_score,
            "Difference": abs(predicted_score - actual_score),
        }
    )

df_model = pd.DataFrame(model_analysis)

# Visualize model prediction vs actual
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Scatter plot: Predicted vs Actual
ax1.scatter(
    df_model["Predicted Score"],
    df_model["Actual Score"],
    s=100,
    alpha=0.7,
    c=range(len(df_model)),
    cmap="viridis",
)
ax1.plot([0, 10], [0, 10], "r--", alpha=0.5, label="Perfect Prediction")

for i, row in df_model.iterrows():
    ax1.annotate(
        row["Progression"].split(" - ")[0],
        (row["Predicted Score"], row["Actual Score"]),
        xytext=(5, 5),
        textcoords="offset points",
        fontsize=9,
    )

ax1.set_xlabel("Predicted Score")
ax1.set_ylabel("Actual Score")
ax1.set_title("Model Prediction Accuracy")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Bar chart: Component contribution
components = ["Harmonic", "Rhythmic", "Technical", "Melodic"]
weights = [model_weights[f"{comp.lower()}_complexity"] for comp in components]

bars = ax2.bar(components, weights, color=["#FF6B6B", "#4ECDC4", "#45B7D1", "#96CEB4"])
ax2.set_ylabel("Weight in Model")
ax2.set_title("Model Feature Weights")
ax2.set_ylim(0, 0.4)

# Add percentage labels on bars
for bar, weight in zip(bars, weights):
    ax2.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.01,
        f"{weight:.0%}",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

plt.tight_layout()
plt.show()

print("\n📊 Model Analysis Results:")
print(df_model.to_string(index=False, float_format="%.2f"))

## Interactive Complexity Explorer

Use the interactive widgets below to explore how different factors affect complexity scores:

In [ ]:
# Create interactive widgets for exploring complexity
def create_complexity_explorer():
    # Widgets for adjusting complexity components
    harmonic_slider = widgets.FloatSlider(
        value=5.0,
        min=0.0,
        max=10.0,
        step=0.1,
        description="Harmonic:",
        style={"description_width": "initial"},
    )

    rhythmic_slider = widgets.FloatSlider(
        value=3.0,
        min=0.0,
        max=10.0,
        step=0.1,
        description="Rhythmic:",
        style={"description_width": "initial"},
    )

    technical_slider = widgets.FloatSlider(
        value=4.0,
        min=0.0,
        max=10.0,
        step=0.1,
        description="Technical:",
        style={"description_width": "initial"},
    )

    melodic_slider = widgets.FloatSlider(
        value=2.0,
        min=0.0,
        max=10.0,
        step=0.1,
        description="Melodic:",
        style={"description_width": "initial"},
    )

    output = widgets.Output()

    def update_complexity(*args) -> None:
        with output:
            output.clear_output(wait=True)

            # Calculate weighted score
            weighted_input = calculate_weighted_input(
                harmonic_slider.value,
                rhythmic_slider.value,
                technical_slider.value,
                melodic_slider.value,
            )

            overall_score = apply_polynomial(weighted_input)

            # Determine skill level
            if overall_score < 2.5:
                skill_level = "Beginner"
                color = "#96CEB4"
            elif overall_score < 5.0:
                skill_level = "Intermediate"
                color = "#FECA57"
            elif overall_score < 7.5:
                skill_level = "Advanced"
                color = "#FF9FF3"
            else:
                skill_level = "Expert"
                color = "#FF6B6B"

            # Create visualization
            fig = make_subplots(
                rows=1,
                cols=2,
                subplot_titles=["Component Breakdown", "Overall Score"],
                specs=[[{"type": "polar"}, {"type": "indicator"}]],
            )

            # Radar chart
            components = ["Harmonic", "Rhythmic", "Technical", "Melodic"]
            values = [
                harmonic_slider.value,
                rhythmic_slider.value,
                technical_slider.value,
                melodic_slider.value,
            ]

            fig.add_trace(
                go.Scatterpolar(
                    r=values + [values[0]],
                    theta=components + [components[0]],
                    fill="toself",
                    fillcolor=color,
                    line=dict(color=color, width=2),
                    opacity=0.7,
                    name="Complexity",
                ),
                row=1,
                col=1,
            )

            fig.update_polars(
                radialaxis=dict(range=[0, 10], tickmode="linear", tick0=0, dtick=2),
                row=1,
                col=1,
            )

            # Gauge chart
            fig.add_trace(
                go.Indicator(
                    mode="gauge+number+delta",
                    value=overall_score,
                    domain={"x": [0, 1], "y": [0, 1]},
                    title={
                        "text": f"Overall Complexity<br><span style='font-size:0.8em'>{skill_level}</span>"
                    },
                    gauge={
                        "axis": {"range": [None, 10]},
                        "bar": {"color": color},
                        "steps": [
                            {"range": [0, 2.5], "color": "lightgreen"},
                            {"range": [2.5, 5.0], "color": "yellow"},
                            {"range": [5.0, 7.5], "color": "orange"},
                            {"range": [7.5, 10], "color": "red"},
                        ],
                        "threshold": {
                            "line": {"color": "black", "width": 4},
                            "thickness": 0.75,
                            "value": overall_score,
                        },
                    },
                ),
                row=1,
                col=2,
            )

            fig.update_layout(
                height=400, showlegend=False, title="Interactive Complexity Analysis"
            )

            fig.show()

            # Display detailed information
            print("\n📊 Detailed Analysis:")
            print(f"Weighted Input: {weighted_input:.2f}")
            print(f"Overall Score: {overall_score:.2f}/10")
            print(f"Skill Level: {skill_level}")
            print("\nComponent Contributions (weighted):")
            for comp, value, weight in zip(components, values, model_weights.values()):
                contribution = value * weight
                print(f"  {comp}: {value:.1f} × {weight:.0%} = {contribution:.2f}")

    # Attach observers
    harmonic_slider.observe(update_complexity, names="value")
    rhythmic_slider.observe(update_complexity, names="value")
    technical_slider.observe(update_complexity, names="value")
    melodic_slider.observe(update_complexity, names="value")

    # Initial update
    update_complexity()

    # Display widgets
    controls = widgets.VBox(
        [
            widgets.HTML("<h3>🎵 Adjust Complexity Components:</h3>"),
            harmonic_slider,
            rhythmic_slider,
            technical_slider,
            melodic_slider,
        ]
    )

    return widgets.VBox([controls, output])


# Display the interactive explorer
explorer = create_complexity_explorer()
display(explorer)

## Real-World Examples and Validation

Let's test the complexity analysis on some well-known musical progressions to validate the model:

In [ ]:
# Famous chord progressions from real songs
famous_progressions = {
    "Let It Be (Beatles)": {
        "chords": [
            composer.Chord(1, 5),
            composer.Chord(5, 5),
            composer.Chord(6, 5),
            composer.Chord(4, 5),
        ],
        "expected_level": "Beginner",
        "tempo": 75.0,
    },
    "Autumn Leaves (Jazz Standard)": {
        "chords": [
            composer.Chord(6, 7),
            composer.Chord(2, 7),
            composer.Chord(5, 7),
            composer.Chord(1, 7),
        ],
        "expected_level": "Intermediate",
        "tempo": 120.0,
    },
    "Giant Steps (Coltrane)": {
        "chords": [
            composer.Chord(1, 7),  # Bmaj7
            composer.Chord(5, 7),  # D7
            composer.Chord(1, 7),  # Gmaj7
            composer.Chord(3, 7),  # Bb7
            composer.Chord(6, 7),  # Ebmaj7
        ],
        "expected_level": "Expert",
        "tempo": 290.0,
    },
}

# Analyze famous progressions
validation_results = []

for name, prog in famous_progressions.items():
    try:
        assessment = engine.assess_difficulty(prog["chords"], prog["tempo"], (4, 4))

        validation_results.append(
            {
                "Song": name,
                "Expected Level": prog["expected_level"],
                "Predicted Level": assessment.skill_level,
                "Overall Score": assessment.overall_score,
                "Confidence": assessment.confidence,
                "Match": prog["expected_level"] == assessment.skill_level,
            }
        )

        print(
            f"✅ {name}: Expected {prog['expected_level']}, Got {assessment.skill_level} (Score: {assessment.overall_score:.2f})"
        )

    except Exception as e:
        print(f"❌ Error analyzing {name}: {e}")

df_validation = pd.DataFrame(validation_results)

# Display validation results
print("\n🎵 Validation Against Famous Progressions:")
if len(validation_results) > 0:
    print(df_validation.to_string(index=False))

    accuracy = df_validation["Match"].mean()
    print(f"\n✅ Model Accuracy: {accuracy:.0%}")

    # Visualize validation results
    fig = px.scatter(
        df_validation,
        x="Expected Level",
        y="Overall Score",
        size="Confidence",
        color="Predicted Level",
        hover_data=["Song"],
        title="Model Validation: Expected vs Predicted Complexity",
    )

    fig.update_layout(
        xaxis_title="Expected Skill Level",
        yaxis_title="Predicted Complexity Score",
        height=500,
    )

    fig.show()
else:
    print("No validation results to display.")

## Summary and Insights

This analysis has revealed key insights about the Composer AI complexity assessment system:

In [ ]:
# Generate summary insights
print("🎯 Key Insights from Complexity Analysis:\n")

# Find most important complexity component
avg_components = df_complexity[["Harmonic", "Rhythmic", "Technical", "Melodic"]].mean()
most_important = avg_components.idxmax()
print(
    f"1. **{most_important} Complexity** is typically the highest component across progressions"
)
print(f"   Average scores: {avg_components.to_dict()}")

# Model weight vs actual importance
print("\n2. **Model Weights vs Actual Importance:**")
for comp in ["Harmonic", "Rhythmic", "Technical", "Melodic"]:
    weight = model_weights[f"{comp.lower()}_complexity"]
    avg_score = avg_components[comp]
    print(f"   {comp}: {weight:.0%} weight, {avg_score:.1f} avg score")

# Skill level distribution
skill_counts = df_complexity["Skill Level"].value_counts()
print("\n3. **Skill Level Distribution:**")
for level, count in skill_counts.items():
    percentage = count / len(df_complexity) * 100
    print(f"   {level}: {count} progressions ({percentage:.0f}%)")

# Model performance
print("\n4. **Model Performance:**")
avg_confidence = df_complexity["Confidence"].mean()
print(f"   Average Confidence: {avg_confidence:.1%}")
print(f"   Validation Accuracy: {accuracy:.0%}")

# Practical recommendations
print("\n🎼 **Practical Recommendations:**")
print("   • For beginners: Focus on simple triads (I-IV-V-I patterns)")
print("   • For intermediates: Introduce seventh chords and moderate voice leading")
print("   • For advanced: Use extended harmonies and complex progressions")
print("   • Consider tempo and time signature changes for additional complexity")

print(
    "\n✨ The Composer AI complexity analysis provides a sophisticated, multi-dimensional"
)
print(
    "   assessment that considers harmonic, rhythmic, technical, and melodic factors"
)
print("   to accurately classify musical difficulty levels.")

## Next Steps

To further explore the Composer AI system:

1. **Check out the next notebook**: `02_chord_progression_suggester.ipynb` - Analyzes the chord suggestion algorithms
2. **Experiment with custom progressions**: Use the interactive explorer above to test your own chord progressions
3. **Modify model parameters**: Try adjusting the polynomial coefficients or feature weights to see their impact
4. **Explore edge cases**: Test progressions with unusual time signatures, extreme tempos, or exotic harmonies

The Composer AI system provides a robust foundation for musical complexity analysis that can be adapted and extended for various musical applications.